# 1. Preparation

In [20]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings('ignore')

from scipy import stats

from sklearn import linear_model
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from math import log

import os
import glob

In [21]:
path = os.path.join(os.getcwd(), "redesign")
ustkamenogorsk_directory = glob.glob(os.path.join(path, "ust-kamenogorsk", "*.csv"))
ust_kamenogorsk = pd.concat((pd.read_csv(csv_file, quotechar='|', parse_dates=['date']) for csv_file in ustkamenogorsk_directory))

### Data cleaning

In [22]:
ust_kamenogorsk.drop(['id', 'date', 'city', 'address',
                      'view_count', 'image_count', 'color'],
                     axis = 1,
                     inplace = True)

In [23]:
ust_kamenogorsk = ust_kamenogorsk[(pd.notnull(ust_kamenogorsk['price']))
                          & (pd.notnull(ust_kamenogorsk['room_count']))
                          & (pd.notnull(ust_kamenogorsk['building_year']))
                          & (pd.notnull(ust_kamenogorsk['area_total']))
                          & (pd.notnull(ust_kamenogorsk['lat']))
                          & (pd.notnull(ust_kamenogorsk['lon']))]

In [24]:
ust_kamenogorsk.head(2)

,lat,lon,price,room_count,is_owner,building_type,building_year,floor,building_floors,area_total,area_living,area_kitchen,renovation,toilet,balcony,balcony_glass,door,phone,inet_type,furniture,floor_type,security,priv_dorm,parking,room_height,complex
0,49.978068,82.582201,6000000,1,1,панельный,1990.0,2.0,5.0,35.0,NaN,NaN,евроремонт,совмещенный,NaN,NaN,металлическая,NaN,NaN,полностью меблирована,NaN,"домофон, видеонаблюдение",NaN,NaN,2.5,NaN
1,49.968811,82.606702,10500000,3,1,панельный,1979.0,1.0,5.0,70.0,68.0,9.0,хорошее,раздельный,NaN,NaN,металлическая,отдельный,NaN,полностью меблирована,дерево,"решетки на окнах, домофон",нет,рядом охраняемая стоянка,NaN,Ворошилова 97


### Data transformation

### Encode categorical variables using dummies

### Split the data

In [14]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(ust_kamenogorsk, train_size = 0.8, random_state = 10)

# 2. Regression Models

### 2.1 Simple Linear Regression

In [16]:
def simple_linear_model(train, test, input_feature):
    regr = linear_model.LinearRegression()
    regr.fit(train.as_matrix(columns = [input_feature]),
             train.as_matrix(columns = ['price']))
    RMSE = mean_squared_error(test.as_matrix(columns = ['price']),
                              regr.predict(test.as_matrix(columns = [input_feature])))**0.5
    return RMSE, regr.intercept_[0], regr.coef_[0][0]

In [17]:
RMSE, w0, w1 = simple_linear_model(train_data, test_data, 'area_total')
print ('RMSE for area_total is: %s ' %RMSE)
print ('intercept is: %s' %w0)
print ('coefficient is: %s' %w1)

RMSE for area_total is: 3743736.13077 
intercept is: -1946365.04762
coefficient is: 211397.468863


In [19]:
input_list = ust_kamenogorsk.columns.values.tolist() # list of column name
input_list.remove('price')
simple_linear_result = pd.DataFrame(columns = ['feature', 'RMSE', 'intercept', 'coefficient'])

# loop that calculate the RMSE of the test data for each input 
for p in input_list:
    RMSE, w1, w0 = simple_linear_model(train_data, test_data, p)
    simple_linear_result = simple_linear_result.append({'feature':p, 'RMSE':RMSE, 'intercept':w0, 'coefficient': w1}
                                                       ,ignore_index=True)
simple_linear_result.sort_values('RMSE').head(10) # display the 10 best estimators

ValueError: could not convert string to float: 'кирпичный'